In [1]:
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

import imaplib
import base64
import os
import email

import sys
import email.header
import datetime

### Permet d'interagir avec OUTLOOK

In [2]:
server = 'outlook.office365.com'
user = "remyadda@msn.com"
password = "Leomessi26"
outputdir = '/Users/remyadda/Desktop/test'
subject = 'CV' #subject line of the emails you want to download attachments from

### Some functions

In [16]:
def connect(server, user, password):
    """Connexion à la boite mail"""
    connexion = imaplib.IMAP4_SSL(server)
    connexion.login(user, password)
    connexion.select()
    return connexion

def close_session(connexion):
    m.close()
    m.logout()
    
# downloads attachment for an email id, which is a unique identifier for an
# email, which is obtained through the msg object in imaplib, see below 
# subjectQuery function. 'emailid' is a variable in the msg object class.

def downloaAttachmentsInEmail(connexion, emailid, outputdir):
    try:
        #si emailid est type byte
        resp, data = connexion.fetch(emailid, "(BODY.PEEK[])")
    except:
        #si emailid est type string
        resp, data = connexion.fetch(str.encode(emailid), "(BODY.PEEK[])")
    email_body = data[0][1]
    mail = email.message_from_bytes(email_body)
    if mail.get_content_maintype() != 'multipart':
        return
    for part in mail.walk():
        if part.get_content_maintype() != 'multipart' and part.get('Content-Disposition') is not None:
            open(outputdir + '/' + part.get_filename(), 'wb').write(part.get_payload(decode=True))

In [4]:
# Utilise ssi on utilise l'objet du mail pour télécharger la PJ
def subjectQuery(connexion, subject, outputdir):
    """Input : Objet du mail
       Output : Telecharge en local la PJ"""
    connexion.select("Inbox")
    typ, msgs = connexion.search(None, '(SUBJECT "' + subject + '")')
    msgs = msgs[0].split()
    for emailid in msgs:
        print(emailid)
        downloaAttachmentsInEmail(connexion, emailid, outputdir)

In [186]:
#TEST : subjectQuery(m, 'CV 3', outputdir)

In [6]:
def get_last_mails(connexion, x):
    """Input: connexion et x derniers ids de mails que l'on souhaite récupérer
       Outputs: liste de x derniers ids des mails"""
    # On récupère la liste de tous les index des mails de INBOX (à confirmer pour "tous")
    rv, data = connexion.search(None, "ALL")
    if rv != 'OK':
        print("No messages found!")
    #Binary to string + clean
    index_list = data[0].decode("utf-8").split()
    index_list.reverse()
    return index_list[:x]

In [7]:
def get_subject(connexion, list_mails_index):
    """Input: connexion et nbe de mails que l'on souhaite récupérer
       Output: """

    connexion.select('INBOX', readonly=True)
    from_list, subject_list = [], []
    for i in list_mails_index:
        typ, msg_data = m.fetch(str(i), '(RFC822)')
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_string(response_part[1].decode("utf-8"))
                for header in [ 'subject', 'to', 'from' ]:
                    print('%-8s: %s' % (header.upper(), msg[header]))
                    if header == 'subject':
                        subject_list.append(msg[header])
                    if header == 'from':
                        from_list.append(msg[header])
    connexion.close()
    connexion.logout()
    return from_list, subject_list

In [8]:
#connexion
m = connect(server, user, password)
#get list of last 5 mails
p = get_last_mails(m, 5)
# get from and subject (not mandatory)
fr, sub = get_subject(m, p)

In [15]:
#TEST (on telecharge les PJ de la liste de mails p)
for i in p:
    downloaAttachmentsInEmail(m, i, "/Users/remyadda/Desktop/AD/Projets/ALFRED/ALFRED_PROJECT/CV_downloading/")